In [1]:
from bs4 import BeautifulSoup
import requests
import re
import numpy as np
from sklearn.base import TransformerMixin
from sklearn.pipeline import make_pipeline, make_union
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import train_test_split

In [2]:
urls_page = BeautifulSoup(requests.get('http://rosettacode.org/wiki/Category:Solutions_by_Programming_Task').content, 'html.parser').find_all('div', {'class' : 'mw-content-ltr'})[-1]

In [4]:
#urls_page.find('div', {'class' : 'mw-content-ltr'})
urls = re.findall(r'href="(.*)" ', str(urls_page))[:15]
pages = [BeautifulSoup(requests.get('http://rosettacode.org'+url).content, 'html.parser') for url in urls]

In [ ]:
#doors = requests.get('http://rosettacode.org/wiki/100_doors')

doors = BeautifulSoup(requests.get('http://rosettacode.org/wiki/100_doors').content, 'html.parser')



In [ ]:
doors.prettify

In [ ]:
langs = ['c', 'csharp', 'lisp', 'clojure', 'haskell', 
         'java', 'javascript', 'ocaml', 'perl', 'php', 
         'python', 'ruby', 'scala', 'scheme']

# lang_dict = {lang : '' for lang in langs}
lang_X = []
lang_y = []

for url in urls:
for string in doors.find_all('pre'):
    for lang in langs:
        if re.search(r'"'+ lang + ' hi', str(string)):
            # lang_dict[lang] = lang_dict[lang] + string.get_text()
            lang_X.append(string)
            lang_y.append(lang)
#print(lang_dict)

In [ ]:
for lang in lang_dict:
    print(lang, " - - ", lang_dict[lang], end='\n\n')

In [ ]:
class FunctionFeaturizer(TransformerMixin):
    def __init__(self, *featurizers):
        self.featurizers = featurizers
        
    def fit(self, X, y=None):
        '''Returns self after transformations'''
        return self
    
    def transform(self, X):
        '''Given a list of original data, return a list of feature vectors'''
        feature_vectors = []
        for x in X:
            feature_vector = [f(x) for f in self.featurizers]
            feature_vectors.append(feature_vector)
        return np.array(feature_vectors)

In [ ]:
features = FunctionFeaturizer(len)

In [ ]:
lang_X_train, lang_X_test, lang_y_train, lang_y_test = train_test_split(lang_X, lang_y)

In [ ]:
pipe = make_pipeline(features, DecisionTreeClassifier())
pipe.fit(lang_X_train, lang_y_train)
pipe.score(lang_X_train, lang_y_train)

In [ ]:
pipe.score(lang_X_test, lang_y_test)